In [3]:
import cv2
from keras.models import load_model
import numpy as np
import os

Using TensorFlow backend.


In [4]:
model = load_model('emojinator.h5')

In [5]:
def get_emojis():
    emojis_folder = 'hand_emo/'
    emojis = []
    for emoji in range(len(os.listdir(emojis_folder))):
        print(emoji)
        emojis.append(cv2.imread(emojis_folder+str(emoji)+'.png', -1))
    return emojis

In [6]:
def keras_predict(model, image):
    processed = keras_process_image(image)
    pred_probab = model.predict(processed)[0]
    pred_class = list(pred_probab).index(max(pred_probab))
    return max(pred_probab), pred_class


def keras_process_image(img):
    image_x = 50
    image_y = 50
    img = cv2.resize(img, (image_x, image_y))
    img = np.array(img, dtype=np.float32)
    img = np.reshape(img, (-1, image_x, image_y, 1))
    return img

In [7]:
def overlay(image, emoji, x,y,w,h):
    emoji = cv2.resize(emoji, (w, h))
    try:
        image[y:y+h, x:x+w] = blend_transparent(image[y:y+h, x:x+w], emoji)
    except:
        pass
    return image

def blend_transparent(face_img, overlay_t_img):
    # Split out the transparency mask from the colour info
    overlay_img = overlay_t_img[:,:,:3] # Grab the BRG planes
    overlay_mask = overlay_t_img[:,:,3:]  # And the alpha plane

    # Again calculate the inverse mask
    background_mask = 255 - overlay_mask

    # Turn the masks into three channel, so we can use them as weights
    overlay_mask = cv2.cvtColor(overlay_mask, cv2.COLOR_GRAY2BGR)
    background_mask = cv2.cvtColor(background_mask, cv2.COLOR_GRAY2BGR)

    # Create a masked out face image, and masked out overlay
    # We convert the images to floating point in range 0.0 - 1.0
    face_part = (face_img * (1 / 255.0)) * (background_mask * (1 / 255.0))
    overlay_part = (overlay_img * (1 / 255.0)) * (overlay_mask * (1 / 255.0))

    # And finally just add them together, and rescale it back to an 8bit integer image
    return np.uint8(cv2.addWeighted(face_part, 255.0, overlay_part, 255.0, 0.0))

In [ ]:
emojis = get_emojis()
cap = cv2.VideoCapture(0)
x, y, w, h = 300, 50, 350, 350

while (cap.isOpened()):
        ret, img = cap.read()
        img = cv2.flip(img, 1)
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        mask2 = cv2.inRange(hsv, np.array([2, 50, 60]), np.array([25, 150, 255]))
        res = cv2.bitwise_and(img, img, mask=mask2)
        gray = cv2.cvtColor(res, cv2.COLOR_BGR2GRAY)
        median = cv2.GaussianBlur(gray, (5, 5), 0)

        kernel_square = np.ones((5, 5), np.uint8)
        dilation = cv2.dilate(median, kernel_square, iterations=2)
        opening = cv2.morphologyEx(dilation, cv2.MORPH_CLOSE, kernel_square)
        ret, thresh = cv2.threshold(opening, 30, 255, cv2.THRESH_BINARY)

        thresh = thresh[y:y + h, x:x + w]
        contours = cv2.findContours(thresh.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)[1]
        if len(contours) > 0:
            contour = max(contours, key=cv2.contourArea)
            if cv2.contourArea(contour) > 2500:
                x, y, w1, h1 = cv2.boundingRect(contour)
                newImage = thresh[y:y + h1, x:x + w1]
                newImage = cv2.resize(newImage, (50, 50))
                pred_probab, pred_class = keras_predict(model, newImage)
                print(pred_class, pred_probab)
                img = overlay(img, emojis[pred_class], 400, 250, 90, 90)

        x, y, w, h = 300, 50, 350, 350
        cv2.imshow("Frame", img)
        cv2.imshow("Contours", thresh)
        k = cv2.waitKey(10)
        if k == 27:
            break

0
1
2
3
4
5
6
7
8
9
10
11
12
11 0.99965096
1 0.9869127
1 0.999841
1 0.99484164
1 0.99484164
1 0.8520436
1 0.8520436
1 0.86214715
1 0.9970319
1 0.9970319
11 0.99935347
11 0.99935347
1 0.9975688
1 0.9975688
1 0.9998796
1 0.9998796
1 0.99999857
1 0.99999857
11 0.9886692
1 0.99658114
1 0.99658114
1 0.99954224
1 0.99954224
1 0.998481
1 0.998481
1 1.0
1 1.0
1 0.9861799
1 0.96674824
1 0.96674824
1 0.9629165
1 0.9629165
11 0.71699685
11 0.71699685
1 0.9649889
1 0.9649889
11 0.89602
11 0.89602
1 0.99987745
1 0.99987745
1 0.99493545
1 0.99493545
1 0.9814556
1 0.9814556
1 0.99848324
1 0.99848324
1 0.9912896
1 0.9912896
1 0.986225
1 0.986225
1 0.99986434
1 0.99986434
1 0.99980956
1 0.99980956
1 0.99891853
1 0.99891853
1 0.9973725
1 0.9973725
1 0.93830407
1 0.93830407
1 0.9998611
1 0.9998611
1 0.9999572
1 0.9999572
1 0.99989235
1 0.99989235
11 0.92154735
11 0.92154735
1 0.99908054
1 0.99908054
1 0.9997931
1 0.9997931
1 0.9999976
1 0.9999976
1 0.9998816
1 0.9998816
1 0.99762124
1 0.99762124
11 0.995

9 0.9999833
9 0.9999999
9 0.9992555
9 0.9992555
9 0.9999796
9 0.99998856
9 0.99997675
9 0.9995883
9 0.9999995
9 0.9999838
9 0.99999964
9 0.99999857
9 0.99999857
9 0.9999989
9 0.9999994
9 0.99983585
9 0.99999654
9 0.9999871
9 0.9998307
9 0.99998677
9 0.9999056
9 0.999998
9 0.999998
9 0.99999964
9 0.99999917
9 0.9999999
9 0.99994147
9 0.9999999
9 0.9999999
9 0.99999905
9 0.99993265
9 0.99999595
9 0.99999976
9 0.9999999
9 0.99986553
9 0.9999869
9 0.999967
9 0.99992645
9 0.99494606
9 0.9999963
9 0.99999964
9 0.9996793
9 0.9944252
9 0.9944252
9 0.999589
9 0.99849033
9 0.9974808
9 0.8806821
9 0.6830406
6 0.81118995
6 0.858718
6 0.83631754
6 0.7654622
5 0.4122174
5 0.4122174
6 0.7060996
6 0.5354384
6 0.61197466
6 0.6391404
6 0.6120747
6 0.41387928
10 0.7061919
10 0.49665675
10 0.28735128
6 0.49478608
6 0.53409535
6 0.46846348
6 0.35997817
9 0.44980973
5 0.99552983
5 0.6156122
10 0.9312921
10 0.9964395
10 0.9994685
10 0.99893945
10 0.9977245
10 0.96555394
10 0.99064386
10 0.99775875
10 0.98134

8 0.99352866
8 0.9398394
8 0.9961738
8 0.9816171
11 0.55391556
8 0.6681631
8 0.9585919
3 0.6028742
3 0.79138917
8 0.6075145
3 0.64991313
3 0.68252337
3 0.9037409
3 0.89728916
8 0.8283958
3 0.8697174
3 0.9154718
3 0.8168461
3 0.72539824
3 0.9234652
3 0.9683548
1 0.6820288
3 0.6711992
3 0.83771306
3 0.898613
3 0.63927394
9 0.37758943
10 0.53772616
3 0.30893508
10 0.35014316
9 0.29359296
9 0.29359296
10 0.41539043
9 0.47048703
9 0.5682723
6 0.39790845
9 0.61583745
9 0.63107526
9 0.7708553
9 0.42494008
9 0.55546236
9 0.6846234
9 0.6846234
9 0.3078841
10 0.45894623
10 0.45954612
9 0.46417162
9 0.38707188
9 0.65567315
10 0.5201
9 0.43563285
9 0.34165692
6 0.7586339
6 0.7586339
6 0.2779491
6 0.29510662
10 0.3386578
6 0.6903116
9 0.22013122
6 0.27197438
2 0.32077894
10 0.27660277
10 0.42665482
9 0.44737482
9 0.44737482
6 0.7183059
6 0.78505105
10 0.6884114
10 0.9548456
10 0.8189469
10 0.6402693
6 0.5791432
6 0.67814785
10 0.9761248
10 0.8189293
10 0.8189293
10 0.5358192
6 0.8499923
6 0.862777
